# **CASE: *TURNOVER* DE FUNCIONÁRIOS**

---

### Instalação de bibliotecas

In [1]:
# # Para cálculo do IV
# install.packages("Information")
# # Para categorização de variáveis
# install.packages("gtools")
# # Para cálculo do VIF
# install.packages("rms")
# # Para cálculo do KS e AUC
# install.packages("ROCR")

#install.packages("tidyverse")
# install.packages("Information")

### Carregamento de bibliotecas

In [2]:
library(tidyverse) # já pré-instalado, se for em Google Colab
library(Information)
library(gtools)
library(rms)
library(ROCR)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Hmisc


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units




### Leitura das bases de dados

In [3]:
dados_turnover_jan <- read.table(file = "../Dados/Turnover_Funcionarios_Jan_23.txt",
                                 sep = "\t",
                                 dec = ".",
                                 header = TRUE)

In [4]:
dados_turnover_jul <- read.table(file = "../Dados/Turnover_Funcionarios_Jul_23.txt",
                                 sep = "\t",
                                 dec = ".",
                                 header = TRUE)

### Tamanho das bases de dados

In [5]:
dim(dados_turnover_jan)

[1] 3869   17

In [6]:
dim(dados_turnover_jul)

[1] 4486   17

### Análise exploratória breve (safra de jan/23)

In [7]:
##### Medidas resumo
summa <- summary(dados_turnover_jan)
View(summa)

 ID_respondente ult_avaliacao_clima qtde_projetos_3m qtde_projetos_6m
 Min.   :   1   Min.   :0.0000      Min.   :0.000    Min.   :2.000   
 1st Qu.: 968   1st Qu.:0.3900      1st Qu.:1.000    1st Qu.:3.000   
 Median :1935   Median :0.6900      Median :2.000    Median :4.000   
 Mean   :1935   Mean   :0.6218      Mean   :1.882    Mean   :3.792   
 3rd Qu.:2902   3rd Qu.:0.8800      3rd Qu.:3.000    3rd Qu.:5.000   
 Max.   :3869   Max.   :1.0000      Max.   :7.000    Max.   :7.000   
                NA's   :217                                          
 qtde_projetos_12m qtde_projetos_24m media_horas_trabalho_3m
 Min.   : 2.00     Min.   : 2.000    Min.   :139.0          
 1st Qu.: 4.00     1st Qu.: 6.000    1st Qu.:155.0          
 Median : 5.00     Median : 8.000    Median :162.0          
 Mean   : 5.31     Mean   : 7.829    Mean   :166.1          
 3rd Qu.: 6.00     3rd Qu.:10.000    3rd Qu.:177.0          
 Max.   :10.00     Max.   :15.000    Max.   :214.0          
             

In [8]:
# Tabela de frequências: departamento
table(dados_turnover_jan$departamento, useNA = "always")
prop.table(table(dados_turnover_jan$departamento, useNA = "always"))


            dados marketing/produto     RH/financeiro        TI/suporte 
              886               697               408               863 
           vendas              <NA> 
             1015                 0 


            dados marketing/produto     RH/financeiro        TI/suporte 
        0.2289997         0.1801499         0.1054536         0.2230551 
           vendas              <NA> 
        0.2623417         0.0000000 

In [9]:
# Tabela de frequências: patamar de salário
table(dados_turnover_jan$patamar_salario, useNA = "always")
prop.table(table(dados_turnover_jan$patamar_salario, useNA = "always"))


01_menor_igual_media       02_acima_media                 <NA> 
                3220                  649                    0 


01_menor_igual_media       02_acima_media                 <NA> 
           0.8322564            0.1677436            0.0000000 

### Análise do poder preditivo das variáveis (IV)

In [10]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [11]:
# Cálculo e exibição dos IV
IV <- create_infotables(data = dados_turnover_jan[,c(2:17)],
                        y = "turnover")
IV$Summary

,Variable,IV
,<chr>,<dbl>
14,departamento,0.27160174710
15,patamar_salario,0.16864940252
9,tempo_empresa,0.08746832259
1,ult_avaliacao_clima,0.07513113611
10,flag_promocao_3m,0.02395402484
7,media_horas_trabalho_6m,0.01505152877
5,qtde_projetos_24m,0.01422702507
6,media_horas_trabalho_3m,0.01032487509
8,media_horas_trabalho_12m,0.00867632135


### Categorização de variáveis com valores *missing* (safra de jan/23)

In [12]:
# Cenários de categorização da variável 'ult_avaliacao_clima', com diferentes quantidades de grupos
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 3), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 4), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 5), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 6), dados_turnover_jan$turnover, useNA = "always"), 1)

             
                      0         1      <NA>
  [0,0.51]    0.7821138 0.2178862 0.0000000
  (0.51,0.83] 0.8221504 0.1778496 0.0000000
  (0.83,1]    0.8717300 0.1282700 0.0000000
  <NA>        0.8248848 0.1751152 0.0000000

             
                      0         1      <NA>
  [0,0.39]    0.7744035 0.2255965 0.0000000
  (0.39,0.69] 0.8110497 0.1889503 0.0000000
  (0.69,0.88] 0.8426230 0.1573770 0.0000000
  (0.88,1]    0.8714286 0.1285714 0.0000000
  <NA>        0.8248848 0.1751152 0.0000000

             
                      0         1      <NA>
  [0,0.31]    0.7720000 0.2280000 0.0000000
  (0.31,0.59] 0.7991632 0.2008368 0.0000000
  (0.59,0.78] 0.8147651 0.1852349 0.0000000
  (0.78,0.92] 0.8777633 0.1222367 0.0000000
  (0.92,1]    0.8614009 0.1385991 0.0000000
  <NA>        0.8248848 0.1751152 0.0000000

              
                       0         1      <NA>
  [0,0.245]    0.7635468 0.2364532 0.0000000
  (0.245,0.51] 0.8003221 0.1996779 0.0000000
  (0.51,0.69]  0.8140704 0.1859296 0.0000000
  (0.69,0.83]  0.8296875 0.1703125 0.0000000
  (0.83,0.93]  0.8780069 0.1219931 0.0000000
  (0.93,1]     0.8656716 0.1343284 0.0000000
  <NA>         0.8248848 0.1751152 0.0000000

In [13]:
# Criação de variável 'ult_avaliacao_clima_cat' com cenário escolhido de categorização
dados_turnover_jan$ult_avaliacao_clima_cat <- as.factor(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 3))

In [14]:
# Criação de categoria 'Vazio' na nova variável 'ult_avaliacao_clima_cat'
levels(dados_turnover_jan$ult_avaliacao_clima_cat) <- c(levels(dados_turnover_jan$ult_avaliacao_clima_cat), "Vazio")
dados_turnover_jan$ult_avaliacao_clima_cat[is.na(dados_turnover_jan$ult_avaliacao_clima_cat)] <- "Vazio"

In [15]:
View(dados_turnover_jan)

ID_respondente,ult_avaliacao_clima,qtde_projetos_3m,qtde_projetos_6m,qtde_projetos_12m,qtde_projetos_24m,media_horas_trabalho_3m,media_horas_trabalho_6m,media_horas_trabalho_12m,tempo_empresa,flag_promocao_3m,flag_promocao_6m,flag_promocao_12m,flag_promocao_vida,departamento,patamar_salario,turnover,ult_avaliacao_clima_cat
<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<fct>
1,0.92,0,5,7,10,183,180,186,2,1,0,0,1,marketing/produto,01_menor_igual_media,0,"(0.83,1]"
2,0.73,3,3,4,6,158,159,154,4,0,0,0,0,vendas,02_acima_media,0,"(0.51,0.83]"
3,0.80,1,3,6,9,171,176,174,3,0,0,0,0,dados,01_menor_igual_media,0,"(0.51,0.83]"
4,0.66,0,4,7,9,160,157,162,10,0,0,0,1,marketing/produto,01_menor_igual_media,0,"(0.51,0.83]"
5,0.06,3,6,7,9,202,192,182,5,0,0,0,1,vendas,01_menor_igual_media,0,"[0,0.51]"
6,0.14,3,3,4,4,146,151,155,4,0,0,0,0,dados,01_menor_igual_media,0,"[0,0.51]"
7,0.99,1,5,5,6,150,158,159,3,0,0,0,0,vendas,01_menor_igual_media,0,"(0.83,1]"
8,NA,1,6,8,11,191,192,180,5,0,0,0,0,dados,01_menor_igual_media,0,Vazio
9,NA,2,5,7,9,154,155,155,6,0,0,0,1,marketing/produto,01_menor_igual_media,0,Vazio


### Separação de conjuntos de treino e teste

In [16]:
# Aletorizando a ordem das linhas da base
set.seed(12345)
dados_turnover_jan = sample_n(dados_turnover_jan, size = nrow(dados_turnover_jan))
# Definindo tamanho do conjunto de treino
tamanho_treino <- floor(nrow(dados_turnover_jan) * 0.7)
# Separando os conjuntos de treino e teste
dados_turnover_treino <- dados_turnover_jan[1:tamanho_treino,]
dados_turnover_teste  <- dados_turnover_jan[(tamanho_treino+1):nrow(dados_turnover_jan),]

In [17]:
# Aletorizando a ordem das linhas da base
set.seed(12345)
dados_turnover_jan = sample_n(dados_turnover_jan, size = nrow(dados_turnover_jan))

In [18]:
# Definindo tamanho do conjunto de treino
tamanho_treino <- floor(nrow(dados_turnover_jan) * 0.7)

In [19]:
# Separando os conjuntos de treino e teste
dados_turnover_treino <- dados_turnover_jan[1:tamanho_treino,]
dados_turnover_teste  <- dados_turnover_jan[(tamanho_treino+1):nrow(dados_turnover_jan),]

In [20]:
View(dados_turnover_treino)

,ID_respondente,ult_avaliacao_clima,qtde_projetos_3m,qtde_projetos_6m,qtde_projetos_12m,qtde_projetos_24m,media_horas_trabalho_3m,media_horas_trabalho_6m,media_horas_trabalho_12m,tempo_empresa,flag_promocao_3m,flag_promocao_6m,flag_promocao_12m,flag_promocao_vida,departamento,patamar_salario,turnover,ult_avaliacao_clima_cat
,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<fct>
1,2541,0.94,3,4,4,5,183,183,175,3,0,0,0,0,dados,01_menor_igual_media,0,"(0.83,1]"
2,971,0.58,1,2,2,7,155,155,149,3,0,0,0,0,vendas,01_menor_igual_media,0,"(0.51,0.83]"
3,903,0.70,2,3,3,4,155,158,158,2,0,0,0,0,vendas,01_menor_igual_media,1,"(0.51,0.83]"
4,2750,0.62,2,4,4,8,156,158,156,4,0,0,0,0,dados,01_menor_igual_media,0,"(0.51,0.83]"
5,2931,0.90,0,4,6,8,147,157,160,2,0,0,0,0,vendas,02_acima_media,0,"(0.83,1]"
6,780,0.68,2,3,5,9,158,155,151,2,0,0,0,0,TI/suporte,01_menor_igual_media,0,"(0.51,0.83]"
7,1037,0.50,3,4,4,5,158,158,167,3,0,0,1,1,dados,01_menor_igual_media,0,"[0,0.51]"
8,739,NA,0,3,3,6,152,155,168,3,0,0,0,0,TI/suporte,01_menor_igual_media,0,Vazio
9,3509,0.05,1,5,5,6,153,163,163,6,0,0,0,1,dados,01_menor_igual_media,0,"[0,0.51]"


### Modelo de regressão logística múltipla

In [21]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     qtde_projetos_24m +
                     media_horas_trabalho_6m +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_1)


Call:
glm(formula = turnover ~ ult_avaliacao_clima_cat + qtde_projetos_24m + 
    media_horas_trabalho_6m + tempo_empresa + flag_promocao_3m + 
    departamento + patamar_salario, family = binomial(link = "logit"), 
    data = dados_turnover_treino)

Coefficients:
                                    Estimate Std. Error z value             Pr(>|z|)    
(Intercept)                        -1.655743   0.710856  -2.329               0.0198 *  
ult_avaliacao_clima_cat(0.51,0.83] -0.287120   0.127124  -2.259               0.0239 *  
ult_avaliacao_clima_cat(0.83,1]    -0.664763   0.138009  -4.817      0.0000014585706 ***
ult_avaliacao_clima_catVazio       -0.135505   0.239861  -0.565               0.5721    
qtde_projetos_24m                   0.022907   0.022958   0.998               0.3184    
media_horas_trabalho_6m             0.001164   0.004243   0.274               0.7837    
tempo_empresa                      -0.247579   0.038764  -6.387      0.0000000001693 ***
flag_promocao_3m      

In [22]:
# Ajuste do modelo 2: retirando 'media_horas_trabalho_6m'
regressao_2 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     qtde_projetos_24m +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_2)


Call:
glm(formula = turnover ~ ult_avaliacao_clima_cat + qtde_projetos_24m + 
    tempo_empresa + flag_promocao_3m + departamento + patamar_salario, 
    family = binomial(link = "logit"), data = dados_turnover_treino)

Coefficients:
                                   Estimate Std. Error z value             Pr(>|z|)    
(Intercept)                        -1.47518    0.26892  -5.486      0.0000000412149 ***
ult_avaliacao_clima_cat(0.51,0.83] -0.28759    0.12710  -2.263               0.0237 *  
ult_avaliacao_clima_cat(0.83,1]    -0.66420    0.13799  -4.813      0.0000014837027 ***
ult_avaliacao_clima_catVazio       -0.13706    0.23977  -0.572               0.5676    
qtde_projetos_24m                   0.02425    0.02243   1.081               0.2796    
tempo_empresa                      -0.24657    0.03857  -6.392      0.0000000001634 ***
flag_promocao_3m                   -0.80341    0.40889  -1.965               0.0494 *  
departamentomarketing/produto       0.87390    0.20741   4.21

In [23]:
# Ajuste do modelo 3: retirando 'qtde_projetos_24m'
regressao_3 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_3)


Call:
glm(formula = turnover ~ ult_avaliacao_clima_cat + tempo_empresa + 
    flag_promocao_3m + departamento + patamar_salario, family = binomial(link = "logit"), 
    data = dados_turnover_treino)

Coefficients:
                                   Estimate Std. Error z value             Pr(>|z|)    
(Intercept)                        -1.29612    0.21087  -6.147       0.000000000792 ***
ult_avaliacao_clima_cat(0.51,0.83] -0.29140    0.12699  -2.295               0.0218 *  
ult_avaliacao_clima_cat(0.83,1]    -0.66284    0.13788  -4.807       0.000001530599 ***
ult_avaliacao_clima_catVazio       -0.14482    0.23971  -0.604               0.5458    
tempo_empresa                      -0.24282    0.03833  -6.335       0.000000000237 ***
flag_promocao_3m                   -0.80565    0.40886  -1.970               0.0488 *  
departamentomarketing/produto       0.87647    0.20740   4.226       0.000023802266 ***
departamentoRH/financeiro           1.10789    0.21903   5.058       0.0000004233

In [24]:
# Avaliação de colinearidade no modelo 3 (é natural que os VIF sejam altos para dummies de uma mesma variável qualitativa)
data.frame(VIF = vif(regressao_3))

,VIF
,<dbl>
"ult_avaliacao_clima_cat(0.51,0.83]",1.279333
"ult_avaliacao_clima_cat(0.83,1]",1.271227
ult_avaliacao_clima_catVazio,1.087210
tempo_empresa,1.026730
flag_promocao_3m,1.006806
departamentomarketing/produto,1.951655
departamentoRH/financeiro,1.778039
departamentoTI/suporte,2.372352
departamentovendas,2.659806


### Aplicação do modelo no conjunto de treino

In [25]:
# Aplicação do modelo no conjunto de treino (criação de uma nova coluna chamada "probabilidade")
dados_turnover_treino$probabilidade <- predict(regressao_3,
                                               dados_turnover_treino,
                                               type = "response")

In [51]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- 0.19
ponto_corte

[1] 0.19

In [52]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_treino$predito <- as.factor(ifelse(dados_turnover_treino$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho no conjunto de treino

In [53]:
# Tabela de classificação
tabela <- table(dados_turnover_treino$turnover, dados_turnover_treino$predito)

In [54]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

[1] 0.6248154

In [55]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

[1] 0.6071749

In [56]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

[1] 0.707113

In [29]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

[1] 0.6144756

In [30]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

[1] 0.590583

In [31]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

[1] 0.7259414

In [32]:
# KS
pred <- prediction(dados_turnover_treino$probabilidade, dados_turnover_treino$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

[1] 0.3184663


In [33]:
# AUC
pred <- prediction(dados_turnover_treino$probabilidade, dados_turnover_treino$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

[1] 0.7101


### Aplicação do modelo no conjunto de teste

In [34]:
# Aplicação do modelo no conjunto de teste (criação de uma nova coluna chamada "probabilidade")
dados_turnover_teste$probabilidade <- predict(regressao_3,
                                              dados_turnover_teste,
                                              type = "response")

In [35]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_teste$predito <- as.factor(ifelse(dados_turnover_teste$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho no conjunto de teste

In [36]:
# Tabela de classificação
tabela <- table(dados_turnover_teste$turnover, dados_turnover_teste$predito)

In [37]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

[1] 0.6149871

In [38]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

[1] 0.5931322

In [39]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

[1] 0.72

In [40]:
# KS
pred <- prediction(dados_turnover_teste$probabilidade, dados_turnover_teste$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

[1] 0.3224454


In [41]:
# AUC
pred <- prediction(dados_turnover_teste$probabilidade, dados_turnover_teste$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

[1] 0.7034313


### Aplicação do modelo na base de validação (safra de jul/23)

In [42]:
# Criação de variável 'ult_avaliacao_clima_cat' com cenário escolhido de categorização
dados_turnover_jul$ult_avaliacao_clima_cat <- as.factor(case_when(dados_turnover_jul$ult_avaliacao_clima <= 0.51 ~ "[0,0.51]",
                                                                  dados_turnover_jul$ult_avaliacao_clima <= 0.83 ~ "(0.51,0.83]",
                                                                  dados_turnover_jul$ult_avaliacao_clima <= 1    ~ "(0.83,1]",
                                                                  is.na(dados_turnover_jul$ult_avaliacao_clima)  ~ "Vazio"))

In [43]:
# Aplicação do modelo no conjunto de teste (criação de uma nova coluna chamada "probabilidade")
dados_turnover_jul$probabilidade <- predict(regressao_3,
                                            dados_turnover_jul,
                                            type = "response")

In [44]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_jul$predito <- as.factor(ifelse(dados_turnover_jul$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho na base de validação

In [45]:
# Tabela de classificação
tabela <- table(dados_turnover_jul$turnover, dados_turnover_jul$predito)

In [46]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

[1] 0.6074454

In [47]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

[1] 0.5849108

In [48]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

[1] 0.705113

In [49]:
# KS
pred <- prediction(dados_turnover_jul$probabilidade, dados_turnover_jul$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

[1] 0.2946877


In [50]:
# AUC
pred <- prediction(dados_turnover_jul$probabilidade, dados_turnover_jul$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

[1] 0.6891257
